In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import glob

## Creación del dataframe con todos los datos por hora

In [2]:
#Función para generar un dataframe con los datos de Energia en kWh 
#y de irradiación en Wh/m2 por cada hora del día

def nuevoFrame(mesActual):
    i = 0
    j = 0
    energia = []
    irradiacion = []
    fechas = []
    horas = []
    temperatura = []
    try:
        while i<len(mesActual):
            fechas.append(mesActual['Fecha'][i])
            horas.append(mesActual['Hora'][i])
            energia.append(mesActual['Potencia [kW]'][i:i+12].sum()/12)
            irradiacion.append(mesActual['Insolacion [W/m2]'][i:i+12].sum()/12)
            temperatura.append(mesActual['Temperatura de modulos [C]'][i:i+12].sum()/12)
            i+=12
        datos = pd.DataFrame({'Fecha':fechas,'Hora':horas,'Energia [kWh]':energia,'Irradiacion [Wh/m2]':irradiacion,'Temperatura de modulos [C]':temperatura})
        return datos
    except:
        pass

In [3]:
# Se leen los diferentes archivos csv que existen en la carpeta actual
archivos = ['Octubre 2019.csv','Noviembre 2019.csv', 'Diciembre 2019.csv', 'Enero 2020.csv', 'Febrero 2020.csv',
            'Marzo 2020.csv','Abril 2020.csv', 'Mayo 2020.csv','Junio 2020.csv', 'Julio 2020.csv', 'Agosto 2020.csv',
             'Septiembre 2020.csv','Octubre 2020.csv','Noviembre 2020.csv', 'Diciembre 2020.csv']
len(archivos)

15

### Nuevo dataframe:

In [4]:
# Se leen los diferentes archivos y se ejecuta sobre cada uno la función 'nuevoFrame'
# Luego se concatenan cada uno de estos dataFrames en uno solo
inicio = 0

for name in archivos:
    try:
        mesActual = pd.read_csv(name)      #Lectura
        mesActual = nuevoFrame(mesActual)  #Ejecución
    except:
        pass
    if inicio == 0:
        todoporHora = mesActual
    else:
        todoporHora = pd.concat([todoporHora,mesActual])
    inicio+=1
todoporHora.set_index(np.array(range(len(todoporHora))),inplace = True)

In [5]:
inicio

15

In [6]:
todoporHora

,Fecha,Hora,Energia [kWh],Irradiacion [Wh/m2],Temperatura de modulos [C]
0,1/10/2019,6:00,2.334167,34.787500,12.970833
1,1/10/2019,7:00,15.220000,242.950833,23.139167
2,1/10/2019,8:00,34.786667,522.285000,35.152500
3,1/10/2019,9:00,36.830000,503.754167,31.743333
4,1/10/2019,10:00,30.043333,377.405833,28.122500
...,...,...,...,...,...
5491,31/12/2020,13:00,3.046667,75.204167,18.377500
5492,31/12/2020,14:00,0.524167,19.589167,14.272500
5493,31/12/2020,15:00,0.206667,11.668333,13.439167
5494,31/12/2020,16:00,0.219167,12.330833,14.020000


In [7]:
dataFrame['Energia [kWh]'].plot()
plt.show()
dataFrame['Irradiacion [Wh/m2]'].plot()

NameError: name 'dataFrame' is not defined

In [ ]:
horas = dataFrame['Hora'].unique()
horas

In [ ]:
irr_prom = []
ene_prom = []

irr_std = []
ene_std = []

for h in horas:
    df = dataFrame[dataFrame['Hora']==h]
    
    energy = df['Energia [kWh]'].mean()
    irradiance  =df['Irradiacion [Wh/m2]'].mean()
    
    energy_std = df['Energia [kWh]'].std()
    irradiance_std  =df['Irradiacion [Wh/m2]'].std()
    
    ene_prom.append(energy)
    irr_prom.append(irradiance)
    
    ene_std.append(energy_std)
    irr_std.append(irradiance_std)


In [ ]:
porHora = pd.DataFrame({'Hora':horas,'Energia Prom [kWh]':ene_prom,'Irradiacion Prom [Wh/m2]':irr_prom,
                        'Energia desv [kWh]': ene_std,'Irradiacion desv [Wh/m2]':irr_std})
porHora.to_csv('Estadisticas por hora.csv')

In [ ]:
todoporHora = pd.read_csv('Todo por hora.csv')
todoporHora.drop('Unnamed: 0',axis = 1,inplace = True)
todoporHora = todoporHora[todoporHora.Hora != '6:00']
todoporHora = todoporHora[todoporHora.Hora != '17:00']
todoporHora.to_excel('Parafiltrar.xlsx', sheet_name='Parafiltrar')

In [ ]:
hora = input('Hora: \nEntre 6:00 y 17:00\n')
plt.scatter(range(len(dataFrame[dataFrame['Hora']==hora])),dataFrame[dataFrame['Hora']==hora]['Irradiacion [Wh/m2]'])

In [ ]:
promedios = pd.read_csv('Estadisticas por hora.csv')
promedios.drop('Unnamed: 0',axis = 1,inplace = True)
promedios

## Filtrado 

In [ ]:
filtro1 = []
filtro2 = []
horas = promedios['Hora']

for i in range(len(promedios)):
    sup = promedios['Irradiacion Prom [Wh/m2]'][i] + promedios['Irradiacion desv [Wh/m2]'][i]
    inf = promedios['Irradiacion Prom [Wh/m2]'][i] - promedios['Irradiacion desv [Wh/m2]'][i]
    
    filtro1.append(sup)
    filtro2.append(inf)
filtro = pd.DataFrame({'Hora':horas,'Superior':filtro1,'Inferior':filtro2})
filtro.set_index('Hora', inplace = True)
filtro

In [ ]:
todoporHora[20:25]

In [ ]:
rango1 = []
rango2 = []
horas = promedios['Hora']
for i in range(len(promedios)):
    lim1 = promedios['Irradiacion Prom [Wh/m2]'][i] - promedios['Irradiacion desv [Wh/m2]'][i]/2
    lim2 = promedios['Irradiacion Prom [Wh/m2]'][i] + promedios['Irradiacion desv [Wh/m2]'][i]/2
    
    rango1.append(lim1)
    rango2.append(lim2)
rangos = pd.DataFrame({'Hora':horas,'Limite Inf':rango1,'Limite Sup':rango2})
rangos.set_index('Hora',inplace = True)
rangos

In [ ]:
m = len(todoporHora)

horas = todoporHora['Hora'].unique()
fechas = todoporHora.Fecha

for h in horas:
    inf = filtro['Inferior'][h]
    sup = filtro['Superior'][h]
    for i in range(len(todoporHora)):
        irr_actual =todoporHora['Irradiacion [Wh/m2]'][i]
        
        if todoporHora['Hora'][i] == h:
            if  irr_actual < inf:
                todoporHora.drop([i],inplace = True)
                
            elif irr_actual > sup:
                todoporHora.drop([i], inplace = True)
                
todoporHora.set_index(np.array(range(len(todoporHora))))

# Clasificación

In [ ]:
todoporHora = pd.read_csv('Para filtrar.csv')

In [ ]:
m = len(todoporHora)

soleados = [0]*m
lluviosos = [0]*m
nubados = [0]*m

horas = todoporHora['Hora'].unique()
fechas = todoporHora.Fecha

for h in horas:
    inf = rangos['Limite Inf'][h]
    sup = rangos['Limite Sup'][h]
    for i in range(len(todoporHora)):
        irr_actual =todoporHora['Irradiacion [Wh/m2]'][i]
        
        if todoporHora['Hora'][i] == h:
            if  irr_actual < inf:
                lluviosos[i] = 1
                
            elif inf < irr_actual < sup:
                nubados[i] = 1
            
            else:
                soleados[i] = 1

clima = pd.DataFrame({'Fecha':fechas,'Hora':todoporHora['Hora'],'Lluvioso':lluviosos, 'Nublado':nubados,'Soleado':soleados})

In [ ]:
clima

In [ ]:
counts = [sum(nubados)/5496, sum(lluviosos)/5496,sum(soleados)/5496]
estado = ['Nublado','Lluvioso','Soleado']

colores = ["#EE6055","#60D394","#AAF683"]

plt.figure(figsize=(10,6))
plt.pie(counts,labels = estado,autopct="%0.1f %%",colors = colores)
plt.title('Porcentaje de horas lluviosas, soleadas y nubladas')
plt.axis('equal')

# Muestra de 5496 datos

In [ ]:
horas = clima.Hora.unique()
n = len(horas)
lluvia = [0]*n
sol = [0]*n
nube = [0]*n
for i in range(len(horas)):
    lluvia[i] = clima[clima['Hora']==horas[i]]['Lluvioso'].sum()
    sol[i] = clima[clima['Hora']==horas[i]]['Soleado'].sum()
    nube[i] = clima[clima['Hora']==horas[i]]['Nublado'].sum()
    
climaHora = pd.DataFrame({'Hora':horas,'Lluvioso':lluvia,'Soleado':sol,'Nublado':nube})
climaHora.set_index('Hora',inplace = True)

In [ ]:
colores = ["#DE6055","#60D394","#AAF683"]
climaHora.plot.bar(figsize = (12,8),grid = True,color = colores)
plt.title('Frecuencia de tipo de clima por hora')
plt.ylabel('Frecuencia')

# DataFrames Finales

In [ ]:
#Función para generar un dataframe con los datos de Energia en kWh 
#y de irradiación en Wh/m2 por cada uno de los datos 

def nuevoFrame(mesActual):
    i = 0
    j = 0
    energia = []
    irradiacion = []
    fechas = []
    horas = []
    try:
        while i<len(mesActual):
            fechas.append(mesActual['Fecha'][i])
            horas.append(mesActual['Hora'][i])
            energia.append(mesActual['Potencia [kW]'][i:i+12].sum()/12)
            irradiacion.append(mesActual['Insolacion [W/m2]'][i:i+12].sum()/12)
            i+=12
        datos = pd.DataFrame({'Fecha':fechas,'Hora':horas,'Energia [kWh]':energia,'Irradiacion [Wh/m2]':irradiacion})
        return datos
    except:
        pass

In [ ]:
#Función para generar un dataframe con los datos de Energia en kWh 
#y de irradiación en Wh/m2 por cada hora del día

def nuevoFrame2(mesActual):
    try:
        fechas = mesActual['Fecha']
        horas = mesActual['Hora']
        energia = mesActual['Potencia [kW]']
        irradiacion = mesActual['Insolacion [W/m2]']
        temperatura = mesActual['Temperatura de modulos [C]']
        datos = pd.DataFrame({'Fecha':fechas,'Hora':horas,'Potencia [kW]':energia,'Irradiacion [W/m2]':irradiacion,
                              'Temperatura de modulos[C]':temperatura})
        return datos
    except:
        pass

In [ ]:
archivos = ['Octubre 2019.csv','Noviembre 2019.csv', 'Diciembre 2019.csv', 'Enero 2020.csv', 'Febrero 2020.csv',
            'Marzo 2020.csv','Abril 2020.csv', 'Mayo 2020.csv','Junio 2020.csv', 'Julio 2020.csv', 'Agosto 2020.csv',
             'Septiembre 2020.csv','Octubre 2020.csv','Noviembre 2020.csv', 'Diciembre 2020.csv']
len(archivos)

In [ ]:
start = 0
for arch in archivos:
    try:
        df = pd.read_csv(arch)
        df = nuevoFrame2(df)
    except:
        pass
    if start == 0:
        dataFinal= df
    else:
        dataFinal= pd.concat([dataFinal,df])
    start+=1
dataFinal.to_csv('DataFinalBeta.csv')

In [ ]:
dataFinal = pd.read_csv('DataFinalBeta.csv')
dataFinal.drop('Unnamed: 0',axis = 1 ,inplace = True)
dataFinal

In [ ]:
n = len(clima)
soleado = []
nublado  = []
lluvioso = []

for i in range(n):
    soleado+=[clima['Soleado'][i]]*12
    lluvioso+=[clima['Lluvioso'][i]]*12
    nublado+=[clima['Nublado'][i]]*12

dataFinal['Lluvioso'] = lluvioso
dataFinal['Nublado'] = nublado
dataFinal['Soleado'] = soleado

In [ ]:
todoporHora['Lluvioso'] = clima.Lluvioso
todoporHora['Nublado'] = clima.Nublado
todoporHora['Soleado'] = clima.Soleado
todoporHora1 = todoporHora
todoporHora.set_index(np.array(range(len(todoporHora))),inplace = True)
todoporHora.to_csv('Para filtrar.csv')

### Días típicos

In [ ]:
datos = pd.read_csv('DataFinalBeta.csv').drop('Unnamed: 0',axis = 1)
datos.to_csv('DataFinalBeta.csv')

In [ ]:
datos

In [ ]:
# Reemplazo de los valores de potencia iguales a 0.00
horas = ['6:00',
 '6:05',
 '6:10',
 '6:15',
 '6:20',
 '6:25',
 '6:30',
 '6:35',
 '6:40',
 '6:45',
 '6:50',
 '6:55',
 '17:00',
 '17:05',
 '17:10',
 '17:15',
 '17:20',
 '17:25',
 '17:30',
 '17:35',
 '17:40',
 '17:45',
 '17:50',
 '17:55']

for h in horas:
        datos = datos[datos.Hora!=h]
datos.set_index(np.array(range(len(datos))),inplace = True)

In [ ]:
datos.to_csv('DatosFinales.csv')

In [ ]:
datos = pd.read_csv('DatosFinales.csv').drop('Unnamed: 0',axis = 1)
datos

In [ ]:
datos_lluviosos = datos[datos.Lluvioso ==1]
datos_nublados = datos[datos.Nublado ==1]
datos_soleados = datos[datos.Soleado ==1]

In [ ]:
horas = datos.Hora.unique()
prom_lluvioso = [0]*len(horas)
prom_nublado = [0]*len(horas)
prom_soleado = [0]*len(horas)

for i in range(len(horas)):
    h_actual = horas[i]
    prom_lluvioso[i] = datos_lluviosos[datos_lluviosos.Hora == h_actual]['Potencia [kW]'].mean()
    prom_nublado[i] = datos_nublados[datos_nublados.Hora == h_actual]['Potencia [kW]'].mean()
    prom_soleado[i] = datos_soleados[datos_soleados.Hora == h_actual]['Potencia [kW]'].mean()

In [ ]:
dias_tipicos = pd.DataFrame({'Hora':horas,'Lluviosos': prom_lluvioso,'Nublados': prom_nublado, 'Soleados': prom_soleado})
dias_tipicos.set_index('Hora', inplace = True)

In [ ]:
dias_tipicos.plot(figsize = (10,8))
plt.ylabel('Potencia [kW]')
plt.title('Curvas de generación de la planta en días típicos Lluviosos, Nublados y Soleados')

### Histogramas de horas lluviosas, nubladas, soleadas

In [ ]:
lluv_por_hora = todoporHora[todoporHora.Lluvioso == 1]
sol_por_hora = todoporHora[todoporHora.Soleado ==1]
nub_por_hora = todoporHora[todoporHora.Nublado ==1]

In [ ]:
horas = todoporHora.Hora.unique()
print(horas)
h = input('Ingrese una hora válida:')
clima = input('Ingrese estado climático: [Lluvioso, Nublado, Soleado]')
print('\nHistograma',clima,'a la hora',h)
if clima =='Lluvioso':
    h_actual = lluv_por_hora[lluv_por_hora.Hora == h ]['Energia [kWh]']
    plt.hist(h_actual,bins=20,ec ='k')
    
elif clima == 'Nublado':
    h_actual = nub_por_hora[nub_por_hora.Hora == h ]['Energia [kWh]']
    plt.hist(h_actual,bins=20,ec ='k')

elif clima =='Soleado':
    h_actual = sol_por_hora[sol_por_hora.Hora == h ]['Energia [kWh]']
    plt.hist(h_actual,bins=20,ec ='k')         

In [ ]:
las12 = todoporHora[todoporHora.Hora=='12:00']
plt.scatter(las12['Irradiacion [Wh/m2]'],las12['Energia [kWh]'])
las12

In [ ]:
h = input('Ingrese una hora válida:')
h_actual = nub_por_hora[nub_por_hora.Hora == h ]
plt.scatter(h_actual['Irradiacion [Wh/m2]'],h_actual['Energia [kWh]'])

In [ ]:
plt.figure(figsize = (20,15))
plt.scatter(lluv_por_hora['Irradiacion [Wh/m2]'],lluv_por_hora['Energia [kWh]'])
plt.scatter(sol_por_hora['Irradiacion [Wh/m2]'],sol_por_hora['Energia [kWh]'])
plt.scatter(nub_por_hora['Irradiacion [Wh/m2]'],nub_por_hora['Energia [kWh]'])
plt.legend(('Lluvioso','Soleado','Nublado'))